Simple idea:

    - We can partition our code / functionality into two streams:
    i) Finding whales / profitable wallets etc.
    ii) Mapping these wallets to something.

Example of i) we already have written some `early_whales` functionality. For ii) we could use `intersect_dict` functionality. e.g. after identifying several wallets of interest (through whatever means) once per day we could run some code to compute the current intersection of their portfolios. How this might play out: we identify 20 wallets that are highly profitable. Some were early pepe whales, for example. Then, once per day, we run some code to compute the intersection of their portfolios. We notice that 8/20 have purchased a coin $\theta$. Maybe $\theta$ is a newly launched coin - this would be of particular interest! Maybe $\theta$ is a "trending" token. And so on.

Anyway, we basically have the functionality to do this (once given the wallets of interest - which we already have a few of).

In [108]:
from api_endpoints.etherscan_api import *
from api_endpoints.covalent_api import *
from etherscan import Etherscan #externally installed package https://github.com/pcko1/etherscan-python

from fastcore.test import *
from fastcore.basics import *

In [109]:
from typing import List

def strip_none(portfolios:List[dict])->List[dict]:
    """Strip the holdings that have no quote
        Inputs: 
            portfolios: list of `portfolio` dictionaries
        Outputs:
            portfolios: list of `portfolio` dictionaries with no `None` quotes
    """

    for portfolio in portfolios:

        stripped_portfolio=[]
        for _holding in portfolio['portfolio']:
            if _holding['quote'] != None:
                stripped_portfolio.append(_holding)
        portfolio['portfolio'] = stripped_portfolio

    return portfolios


def strip_dust(portfolios:List[dict])->List[dict]:
    """Strip the holdings that are small
        Inputs: 
            portfolios: list of `portfolio` dictionaries
        Outputs:
            portfolios: list of `portfolio` dictionaries with no `None` quotes
    """

    for portfolio in portfolios:

        stripped_portfolio=[]
        for _holding in portfolio['portfolio']:
            if float(_holding['quote']) >= 0.01:
                stripped_portfolio.append(_holding)
        portfolio['portfolio'] = stripped_portfolio

    return portfolios

This code seems to do what we want. Just need to wrap it into a nice class / function.

In [110]:
class WalletListToIntersectDict:
    #TODO: needs a better name perhaps?
    """Wrapper to compute the `intersect_dict` given a list of wallets. Basically a map to the `intersect_dict`        
        Inputs:
            cov_api: Covalent_Api instance
            n_lst: list of wallets, of the form [{'address':wallet_1},{'address':wallet_2}]
            chainName: e.g. 'eth-mainnet'
            date: e.g. '2023-09-24'
            quote_currency: e.g. USD
    """

    def __init__(self, cov_api, n_lst, chainName,date,quote_currency="USD"): 
        store_attr()
        
        self.intersect_dict = self.get_data()
        
    def get_data(self):
        self.portfolios = cov_api.get_holders_portfolios(n_lst=self.n_lst,chainName=self.chainName,date=self.date)
        self.portfolios = strip_none(self.portfolios)
        self.portfolios = strip_dust(self.portfolios)
        self.union_portfolios  = union_top_n(self.portfolios)#i.e. all the tokens ("union")
        self.intersect_dict = intersection_count(self.portfolios,self.union_portfolios)

        return self.intersect_dict

In [ ]:
###Setup
chainName='eth-mainnet'
cov_api = Covalent_Api() #might need to pass an API key (different to default -- see __init__ of class)
###

#Inputs:
wallet_1 = '0xaf2358e98683265cbd3a48509123d390ddf54534' #this guy is an early pepe whale
wallet_2 = '0x711281c1b26aaed86e40e4caaf76c1962b45e161' #this guy is an early pepe whale
n_lst = [{'address':wallet_1},{'address':wallet_2}]
#wallet_1 >> wallet_2, but yeah.
date = '2023-09-24' #todays date. Want this to automatically gets todays date!

#Out:
walletlist_to_intersect_dict = WalletListToIntersectDict(cov_api,n_lst,chainName,date) 
intersect_dict = walletlist_to_intersect_dict.intersect_dict

#now update token holders info by getting their whole portfolios on the given `date`. We take the list output from `get_token_holders` and update it with the portfolio info.
#date = '2023-09-24' #todays date. Want this to automatically gets todays date!
# portfolios = cov_api.get_holders_portfolios(n_lst=n_lst,chainName=chainName,date=date)
# portfolios = strip_none(portfolios)
# portfolios = strip_dust(portfolios)

# union_portfolios  = union_top_n(portfolios)#i.e. all the tokens ("union")
# intersect_dict = intersection_count(portfolios,union_portfolios)


In [112]:
#Test:
for _portfolio in walletlist_to_intersect_dict.portfolios:
    for _holding in _portfolio['portfolio']:
        test_ne(_holding['quote'],None) #check there are no `None` values of holdings.
        assert float(_holding['quote'])>=0.01, f"Expected {_holding['quote']} to be greater than 0.01"

So, after stripping `None` and `dust` from portfolios we are left with the fact that of these two wallets (which are in our list of early pepe whales), they both hold (as of `date` above, which is now in the past):

In [76]:
token_list = contract_name_if_k_holders(walletlist_to_intersect_dict.intersect_dict,2)
token_list

[('Ether', '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
 ('Ushi', '0x6dca182ac5e3f99985bc4ee0f726d6472ab1ec55'),
 ('HarryPotterObamaSonic10Inu', '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9'),
 ('Pepe', '0x6982508145454ce325ddbe47a25d4ec3d2311933'),
 ('USD Coin', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48')]

Clearly it is also of interest *how much* they currently have in each of these tokens. I guess in terms of % of their portfolio and also in absolute terms. Hence we need a nice function to extract this info easily:

In [104]:
def get_intersect_values(token_list,portfolios):
    """Given a list of tokens, and a list of portfolios, return a dictionary of the token values for each portfolio.
        Inputs: 
                token_list: list of tokens, e.g. [('Ether', '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
                            ('Ushi', '0x6dca182ac5e3f99985bc4ee0f726d6472ab1ec55')].
                            So, for example the return value of `contract_name_if_k_holders` function.
                portfolios: list of `portfolio` dictionaries
        Outputs:
                token_list_dict: dictionary, where keys are the portfolio addresses, and values are dictionaries giving the amount of each token
                                 in USD that they hold.
    """

    token_list_dict={}
    for _portfolio in portfolios:
        token_list_dict[_portfolio['address']] = {}
        for _token in token_list:
            for _holding in _portfolio['portfolio']:
                if _holding['contract_address'] == _token[1]: #second element of tuple is the contract address
                    token_list_dict[_portfolio['address']][_token] = _holding['quote']
    
    return token_list_dict

def print_intersect_values(token_list_dict,portfolios):
    "Function to print the values of the tokens in each portfolio."

    for _portfolio in portfolios:
        print(f"Portfolio: {_portfolio['address']}")
        for _token in token_list_dict[_portfolio['address']]:
            print(f"\tToken: {_token[0]}")
            print(f"\t\tQuote: ${token_list_dict[_portfolio['address']][_token]}")
            print(f"\t\tPercentage of portfolio: {token_list_dict[_portfolio['address']][_token]/_portfolio['portfolio_sum'] * 100:.2f}%")
            


How to use `get_intersect_values` and `print_intersect_values`

In [105]:
token_list_dict = get_intersect_values(token_list,portfolios)
print_intersect_values(token_list_dict,portfolios)

Portfolio: 0xaf2358e98683265cbd3a48509123d390ddf54534
	Token: Ether
		Quote: $358162.8
		Percentage of portfolio: 7.88%
	Token: Ushi
		Quote: $4.5941653
		Percentage of portfolio: 0.00%
	Token: HarryPotterObamaSonic10Inu
		Quote: $7.923421
		Percentage of portfolio: 0.00%
	Token: Pepe
		Quote: $1704826.5
		Percentage of portfolio: 37.51%
	Token: USD Coin
		Quote: $49.369713
		Percentage of portfolio: 0.00%
Portfolio: 0x711281c1b26aaed86e40e4caaf76c1962b45e161
	Token: Ether
		Quote: $34805.703
		Percentage of portfolio: 4.69%
	Token: Ushi
		Quote: $0.1186439
		Percentage of portfolio: 0.00%
	Token: HarryPotterObamaSonic10Inu
		Quote: $2.0857906
		Percentage of portfolio: 0.00%
	Token: Pepe
		Quote: $0.15355708
		Percentage of portfolio: 0.00%
	Token: USD Coin
		Quote: $679467.4
		Percentage of portfolio: 91.52%
